In [1]:
from rchess import Board
import numpy as np

In [4]:
#8cefbf3a77078a08, 8cefbf3a77078a08
board = Board()
#board.push_uci("e2e4")
print(board.to_string())


  +---+---+---+---+---+---+---+---+
  | r | n | b | q | k | b | n | r | 8
  +---+---+---+---+---+---+---+---+
  | p | p | p | p | p | p | p | p | 7
  +---+---+---+---+---+---+---+---+
  |   |   |   |   |   |   |   |   | 6
  +---+---+---+---+---+---+---+---+
  |   |   |   |   |   |   |   |   | 5
  +---+---+---+---+---+---+---+---+
  |   |   |   |   |   |   |   |   | 4
  +---+---+---+---+---+---+---+---+
  |   |   |   |   |   |   |   |   | 3
  +---+---+---+---+---+---+---+---+
  | P | P | P | P | P | P | P | P | 2
  +---+---+---+---+---+---+---+---+
  | R | N | B | Q | K | B | N | R | 1
  +---+---+---+---+---+---+---+---+
    a   b   c   d   e   f   g   h

Fen: rnbqkbnr/pppppppp/8/8/8/8/PPPPPPPP/RNBQKBNR w KQkq - 0 1
Key: 8cefbf3a77078a08


In [11]:
from utils import convert_u64_to_np

image, hash = board.history_image(False)
image_np = convert_u64_to_np(image)
print(image_np.shape)
print(image_np[0, 6, :, :])

(1, 117, 8, 8)
[[0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0.]
 [1. 1. 1. 1. 1. 1. 1. 1.]
 [0. 0. 0. 0. 0. 0. 0. 0.]]


In [5]:
images = [image, image]
images_np = convert_u64_to_np(images)
images_np.shape

(2, 117, 8, 8)

In [28]:
import random
import time
t1 = time.time()
board = Board()
winner = None
moves_played = []
while board.ply() < 100:
    terminal, winner = board.is_terminal()
    if terminal:
        break
    moves = board.legal_moves()
    move = moves[random.randint(0, len(moves) - 1)]
    moves_played.append(move)
    board.push_uci(move)
t2 = time.time()
print(t2 - t1, board.ply())
print(board.to_string())
print("WHITE") if winner == 1 else print("BLACK") if winner == 0 else print("DRAW")
print(board.outcome_str())

0.002259969711303711 100

  +---+---+---+---+---+---+---+---+
  |   |   |   |   |   |   | n |   | 8
  +---+---+---+---+---+---+---+---+
  |   | B | k |   |   |   |   | p | 7
  +---+---+---+---+---+---+---+---+
  | P |   |   |   |   | p | P | b | 6
  +---+---+---+---+---+---+---+---+
  | P | p |   |   |   |   | p |   | 5
  +---+---+---+---+---+---+---+---+
  |   |   |   | p | P | p | P |   | 4
  +---+---+---+---+---+---+---+---+
  |   |   |   | P |   | K |   | P | 3
  +---+---+---+---+---+---+---+---+
  |   |   |   | r |   | R |   |   | 2
  +---+---+---+---+---+---+---+---+
  |   |   |   |   |   |   |   |   | 1
  +---+---+---+---+---+---+---+---+
    a   b   c   d   e   f   g   h

Fen: 6n1/1Bk4p/P4pPb/Pp4p1/3pPpP1/3P1K1P/3r1R2/8 w - - 0 51
Key: 834df0ac73db794b
DRAW
InProgress


In [29]:
import chess as pychess
import random
import time
t1 = time.time()
board = pychess.Board()
moves_played = []
while board.ply() < 100:
    if board.is_game_over(claim_draw=True):
        break
    moves = [move.uci() for move in board.legal_moves]
    move = moves[random.randint(0, len(moves) - 1)]
    moves_played.append(move)
    board.push_uci(move)
t2 = time.time()
print(t2 - t1, board.ply())
print(board)
print(board.result())

0.0724029541015625 100
. . k . B . . .
p . . . . . . .
. . . R . . . .
. . . . . . P p
. . . . . . . P
. . P p . b . q
P . . K . B . .
. r . . . . N n
*


In [20]:
moves_played[-1]

'f4e3'

In [9]:
moves_played

['a2a4',
 'g7g5',
 'b2b4',
 'e7e5',
 'c1a3',
 'g8h6',
 'f2f4',
 'b8c6',
 'b4b5',
 'f8c5',
 'h2h3',
 'e5f4',
 'b5c6',
 'd8f6',
 'c6b4',
 'a7a5',
 'd2d3',
 'f6e7',
 'a3b2',
 'e7f8',
 'b1a3',
 'c6e7']

In [2]:
board = Board()
board.push_uci("a2a4")
board.push_uci("g7g5")
board.push_uci("b2b4")
board.push_uci("e7e5")
board.push_uci("c1a3")
board.push_uci("g8h6")
board.push_uci("f2f4")
board.push_uci("b8c6")
print(board.to_string())


  +---+---+---+---+---+---+---+---+
  | r |   | b | q | k | b |   | r | 8
  +---+---+---+---+---+---+---+---+
  | p | p | p | p |   | p |   | p | 7
  +---+---+---+---+---+---+---+---+
  |   |   | n |   |   |   |   | n | 6
  +---+---+---+---+---+---+---+---+
  |   |   |   |   | p |   | p |   | 5
  +---+---+---+---+---+---+---+---+
  | P | P |   |   |   | P |   |   | 4
  +---+---+---+---+---+---+---+---+
  | B |   |   |   |   |   |   |   | 3
  +---+---+---+---+---+---+---+---+
  |   |   | P | P | P |   | P | P | 2
  +---+---+---+---+---+---+---+---+
  | R | N |   | Q | K | B | N | R | 1
  +---+---+---+---+---+---+---+---+
    a   b   c   d   e   f   g   h

Fen: r1bqkb1r/pppp1p1p/2n4n/4p1p1/PP3P2/B7/2PPP1PP/RN1QKBNR w KQkq - 1 5
Key: 9347ef5f29b6ad6c
